In [ ]:
import os
import json
import shutil
import csv
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk

class ImageAnnotator:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Annotator")
        
        # Directory paths for saving annotations
        self.base_dir = "X:\\NSF-B2-project"
        self.yolo_dir = os.path.join(self.base_dir, "Annotated-Training-Images-For-YOLO")
        self.fast_rcnn_dir = os.path.join(self.base_dir, "Annotated-Training-Images-For-Fast-RCNN")
        self.csv_dir = os.path.join(self.base_dir, "Annotation-CSV")
        
        # Create directories if they don't exist
        for directory in [self.yolo_dir, self.fast_rcnn_dir, self.csv_dir]:
            os.makedirs(directory, exist_ok=True)

        # Initialize variables
        self.image_folder = ""
        self.images = []
        self.current_image_index = 0
        self.bboxes = []  # List to store bounding boxes for current image
        self.bbox_start = None
        self.bbox_end = None
        self.rect_id = None
        self.image_loaded = False
        self.img_width = 0
        self.img_height = 0
        
        # Add class mapping for YOLO format
        self.class_mapping = {
            "People": 0,
            "Encampments": 1,
            "Homeless Cart": 2,
            "Homeless Bike": 3
        }
        
        # Create labels.txt file for YOLO
        with open(os.path.join(self.yolo_dir, "labels.txt"), "w") as f:
            for class_name in self.class_mapping.keys():
                f.write(f"{class_name}\n")
        
        # Subclass colors
        self.subclass_colors = {
            "Encampments": "blue",
            "People": "red",
            "Homeless Cart": "green",
            "Homeless Bike": "purple"
        }
        
        # Current selected subclass
        self.current_subclass = "Encampments"
        
        # Create main container frames
        self.subclass_frame = tk.Frame(root)
        self.subclass_frame.pack(pady=5)
        
        self.button_frame = tk.Frame(root)
        self.button_frame.pack(pady=5)
        
        self.canvas = tk.Canvas(root, width=800, height=600)
        self.canvas.pack(pady=10)
        
        # Create subclass buttons
        for idx, subclass in enumerate(self.subclass_colors.keys()):
            btn = tk.Button(
                self.subclass_frame,
                text=subclass,
                command=lambda sc=subclass: self.set_subclass(sc),
                width=15,
                height=2,
                bg=self.subclass_colors[subclass],
                fg='white'
            )
            btn.grid(row=0, column=idx, padx=10)
        
        # Create control buttons
        buttons = [
            ("Load Images", self.load_images),
            ("Skip", self.skip_image),
            ("Previous", self.previous_image),
            ("Next", self.next_image)
        ]
        
        for idx, (text, command) in enumerate(buttons):
            btn = tk.Button(self.button_frame, text=text, command=command, width=15, height=2)
            btn.grid(row=0, column=idx, padx=30)

        # Bind canvas events
        self.canvas.bind("<ButtonPress-1>", self.on_button_press)
        self.canvas.bind("<B1-Motion>", self.on_mouse_drag)
        self.canvas.bind("<ButtonRelease-1>", self.on_button_release)
        
        # Create CSV file if it doesn't exist
        self.csv_file = os.path.join(self.csv_dir, "annotations.csv")
        if not os.path.exists(self.csv_file):
            with open(self.csv_file, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(["filename", "file_attributes", "region_count", "region_id", 
                               "region_shape_attributes", "region_attributes"])

    def set_subclass(self, subclass):
        self.current_subclass = subclass
        # Update button appearances
        for child in self.subclass_frame.winfo_children():
            if child['text'] == subclass:
                child.config(relief=tk.SUNKEN)
            else:
                child.config(relief=tk.RAISED)

    def get_all_images(self, directory):
        """Recursively get all images from directory and its subdirectories"""
        image_files = []
        for root, _, files in os.walk(directory):
            for file in files:
                if file.lower().endswith(('jpg', 'jpeg', 'png')):
                    # Store full path relative to the base directory
                    relative_path = os.path.relpath(os.path.join(root, file), directory)
                    image_files.append(relative_path)
        return image_files

    def load_images(self):
        self.image_folder = filedialog.askdirectory()
        if self.image_folder:
            self.images = self.get_all_images(self.image_folder)
            if not self.images:
                messagebox.showwarning("Warning", "No images found in the selected folder.")
                return
            self.current_image_index = 0
            self.display_image()

    def display_image(self):
        if self.current_image_index < len(self.images):
            image_path = os.path.join(self.image_folder, self.images[self.current_image_index])
            try:
                self.img = Image.open(image_path)
                self.tk_image = ImageTk.PhotoImage(self.img)
                self.canvas.delete("all")
                self.canvas.create_image(0, 0, anchor="nw", image=self.tk_image)
                self.img_width, self.img_height = self.img.size
                self.bbox_start = None
                self.bbox_end = None
                self.bboxes = []  # Now stores tuples of (bbox, subclass)
                self.image_loaded = True
            except Exception as e:
                messagebox.showerror("Error", f"Could not open image: {e}")
                self.image_loaded = False
        else:
            self.end_annotation()

    def on_button_press(self, event):
        if self.image_loaded:
            self.bbox_start = (event.x, event.y)

    def on_button_release(self, event):
        if self.image_loaded and self.bbox_start:
            self.bbox_end = (event.x, event.y)
            
            # Clamp coordinates to image boundaries
            x1 = max(0, min(self.bbox_start[0], self.img_width))
            y1 = max(0, min(self.bbox_start[1], self.img_height))
            x2 = max(0, min(self.bbox_end[0], self.img_width))
            y2 = max(0, min(self.bbox_end[1], self.img_height))

            # Ensure coordinates are in the correct order
            bbox = (min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2))
            # Store both bbox coordinates and subclass
            self.bboxes.append((bbox, self.current_subclass))

            # Draw permanent rectangle with subclass color
            self.canvas.create_rectangle(
                bbox[0], bbox[1], bbox[2], bbox[3],
                outline=self.subclass_colors[self.current_subclass]
            )
            print(f"Added {self.current_subclass} bounding box: {bbox}")
            self.bbox_start = None
            self.bbox_end = None

    def on_mouse_drag(self, event):
        if self.image_loaded and self.bbox_start:
            if self.rect_id:
                self.canvas.delete(self.rect_id)
            self.bbox_end = (event.x, event.y)
            self.rect_id = self.canvas.create_rectangle(
                self.bbox_start[0], self.bbox_start[1],
                event.x, event.y,
                outline=self.subclass_colors[self.current_subclass]
            )

    def save_annotations(self):
        image_filename = self.images[self.current_image_index]
        img_path = os.path.join(self.image_folder, image_filename)

        # Create necessary subdirectories
        for target_dir in [self.yolo_dir, self.fast_rcnn_dir, self.csv_dir]:
            target_subdir = os.path.dirname(os.path.join(target_dir, image_filename))
            os.makedirs(target_subdir, exist_ok=True)
            shutil.copy2(img_path, os.path.join(target_dir, image_filename))

        # Save YOLO format with class information
        yolo_path = os.path.join(self.yolo_dir, f"{os.path.splitext(image_filename)[0]}.txt")
        with open(yolo_path, "w") as yolo_file:
            for bbox, subclass in self.bboxes:
                # Get class id from mapping
                class_id = self.class_mapping[subclass]
                
                # Calculate YOLO format coordinates (normalized)
                x_center = ((bbox[0] + bbox[2]) / 2) / self.img_width
                y_center = ((bbox[1] + bbox[3]) / 2) / self.img_height
                width = abs(bbox[2] - bbox[0]) / self.img_width
                height = abs(bbox[3] - bbox[1]) / self.img_height
                
                # Write class id and coordinates
                yolo_file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

        # Save Pascal VOC XML format
        fast_rcnn_path = os.path.join(self.fast_rcnn_dir, f"{os.path.splitext(image_filename)[0]}.xml")
        xml_content = f"""<annotation>
    <folder>{os.path.dirname(image_filename)}</folder>
    <filename>{os.path.basename(image_filename)}</filename>
    <size>
        <width>{self.img_width}</width>
        <height>{self.img_height}</height>
        <depth>3</depth>
    </size>"""
        
        for bbox, subclass in self.bboxes:
            xml_content += f"""
    <object>
        <name>Homeless_{subclass}</name>
        <bndbox>
            <xmin>{int(bbox[0])}</xmin>
            <ymin>{int(bbox[1])}</ymin>
            <xmax>{int(bbox[2])}</xmax>
            <ymax>{int(bbox[3])}</ymax>
        </bndbox>
    </object>"""
        
        xml_content += "\n</annotation>"
        with open(fast_rcnn_path, "w") as xml_file:
            xml_file.write(xml_content)

        # Get total number of regions for this image
        total_regions = len(self.bboxes)

        # Save CSV annotations
        for idx, (bbox, subclass) in enumerate(self.bboxes):
            region_shape_attributes = {
                "name": "rect",
                "x": int(bbox[0]),
                "y": int(bbox[1]),
                "width": int(bbox[2] - bbox[0]),
                "height": int(bbox[3] - bbox[1])
            }
            
            # Correct format for region_attributes
            region_attributes = f'{{"Homeless":"{subclass}"}}'
            
            with open(self.csv_file, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    image_filename,
                    "{}",
                    total_regions,
                    idx,
                    json.dumps(region_shape_attributes),
                    region_attributes
                ])

        print(f"Saved annotations for {image_filename} in all formats.")

    def skip_image(self):
        if self.image_loaded:
            self.current_image_index += 1
            if self.current_image_index < len(self.images):
                self.display_image()
            else:
                self.end_annotation()

    def previous_image(self):
        if self.image_loaded and self.current_image_index > 0:
            self.current_image_index -= 1
            self.display_image()

    def next_image(self):
        if self.image_loaded:
            if self.bboxes:
                self.save_annotations()
            self.current_image_index += 1
            if self.current_image_index < len(self.images):
                self.display_image()
            else:
                self.end_annotation()

    def end_annotation(self):
        messagebox.showinfo("Done", "All images have been annotated. You can load a new folder.")
        self.image_folder = ""
        self.images = []
        self.current_image_index = 0
        self.bboxes = []
        self.image_loaded = False
        self.canvas.delete("all")

if __name__ == "__main__":
    root = tk.Tk()
    app = ImageAnnotator(root)
    root.mainloop()

Added Homeless Cart bounding box: (393, 356, 466, 452)
Added Encampments bounding box: (453, 357, 518, 449)
Added Encampments bounding box: (541, 337, 638, 455)
Saved annotations for 2022_10_0150kASA6P1e7GawLI9OoQ_90.jpeg in all formats.
